Consolidate and export background and shading

In [1]:
%load_ext autoreload
%autoreload 2

import zarr
import dask.array as da
import pandas as pd
import numpy as np
from scipy import stats, ndimage

from pathlib import Path
from bsccm import BSCCM
from numcodecs import Blosc


home = str(Path.home())
data_root = home + '/leukosight_data/'


backgrounds = zarr.open(data_root + 'backgrounds.zarr' , mode='r')

backgrounds = zarr.open(data_root + 'backgrounds_single.zarr' , mode='r')



slidescan_db = pd.read_csv('slidescan_db.csv')



In [6]:
export_root = home + '/BSCCM_local/BSCCM-coherent/'
export_file = zarr.open(export_root + 'BSCCM_shading.zarr')

In [7]:

#per flatfield group led array backgrounds
for channel_name in   led_channels: 
    print('{}'.format(channel_name))
# [    'Brightfield', 'DF_50','DF_50_Bottom', 'DF_50_Right','DF_55','DF_60','DF_60_Bottom','DF_60_Right','DF_65',
#     'DF_70','DF_70_Bottom','DF_70_Right','DF_75','DF_80','DF_80_Bottom','DF_80_Right', 'DF_85','DF_90',
#     'DPC_Bottom','DPC_Left','DPC_Right','DPC_Top', 'LED119']:

    for percentile in [5, 10, 20, 40, 50]:
        export_file.create_dataset(name='{}/{}_percentile'.format(channel_name, percentile),
#             data=backgrounds['Slidescan 1-0_1']['{}_{}_percentile'.format(channel_name, percentile)],
            data=backgrounds['Slide 2-0d Single LEDs and fluorescence region 1_1']['{}_{}_percentile'.format(channel_name, percentile)],                                   
                                  compressor=Blosc(cname='zstd', clevel=9, shuffle=Blosc.SHUFFLE))
    

led_1
led_2
led_5
led_6
led_7
led_8
led_9
led_13
led_14
led_15
led_16
led_17
led_18
led_19
led_20
led_21
led_22
led_23
led_24
led_25
led_26
led_29
led_30
led_31
led_32
led_33
led_34
led_35
led_36
led_37
led_38
led_39
led_40
led_41
led_42
led_43
led_44
led_45
led_46
led_49
led_50
led_51
led_52
led_53
led_54
led_55
led_56
led_57
led_58
led_59
led_60
led_61
led_62
led_63
led_64
led_65
led_66
led_67
led_68
led_69
led_70
led_71
led_72
led_73
led_74
led_75
led_76
led_77
led_78
led_79
led_80
led_81
led_82
led_85
led_86
led_87
led_88
led_89
led_90
led_91
led_92
led_93
led_94
led_95
led_96
led_97
led_98
led_99
led_100
led_101
led_102
led_103
led_104
led_105
led_106
led_107
led_108
led_109
led_110
led_111
led_112
led_113
led_114
led_115
led_116
led_117
led_118
led_119
led_121
led_122
led_123
led_124
led_125
led_126
led_127
led_128
led_129
led_130
led_131
led_132
led_133
led_134
led_135
led_136
led_137
led_138
led_139
led_140
led_141
led_142
led_143
led_144
led_145
led_146
led_147
led_148
led_149

In [5]:
led_channels = [
    'led_1',
'led_2',
'led_5',
'led_6',
'led_7',
'led_8',
'led_9',
'led_10',
'led_13',
'led_14',
'led_15',
'led_16',
'led_17',
'led_18',
'led_19',
'led_20',
'led_21',
'led_22',
'led_23',
'led_24',
'led_25',
'led_26',
'led_29',
'led_30',
'led_31',
'led_32',
'led_33',
'led_34',
'led_35',
'led_36',
'led_37',
'led_38',
'led_39',
'led_40',
'led_41',
'led_42',
'led_43',
'led_44',
'led_45',
'led_46',
'led_49',
'led_50',
'led_51',
'led_52',
'led_53',
'led_54',
'led_55',
'led_56',
'led_57',
'led_58',
'led_59',
'led_60',
'led_61',
'led_62',
'led_63',
'led_64',
'led_65',
'led_66',
'led_67',
'led_68',
'led_69',
'led_70',
'led_71',
'led_72',
'led_73',
'led_74',
'led_75',
'led_76',
'led_77',
'led_78',
'led_79',
'led_80',
'led_81',
'led_82',
'led_85',
'led_86',
'led_87',
'led_88',
'led_89',
'led_90',
'led_91',
'led_92',
'led_93',
'led_94',
'led_95',
'led_96',
'led_97',
'led_98',
'led_99',
'led_100',
'led_101',
'led_102',
'led_103',
'led_104',
'led_105',
'led_106',
'led_107',
'led_108',
'led_109',
'led_110',
'led_111',
'led_112',
'led_113',
'led_114',
'led_115',
'led_116',
'led_117',
'led_118',
'led_119',
'led_121',
'led_122',
'led_123',
'led_124',
'led_125',
'led_126',
'led_127',
'led_128',
'led_129',
'led_130',
'led_131',
'led_132',
'led_133',
'led_134',
'led_135',
'led_136',
'led_137',
'led_138',
'led_139',
'led_140',
'led_141',
'led_142',
'led_143',
'led_144',
'led_145',
'led_146',
'led_147',
'led_148',
'led_149',
'led_150',
'led_151',
'led_152',
'led_153',
'led_154',
'led_155',
'led_156',
'led_157',
'led_158',
'led_159',
'led_160',
'led_161',
'led_162',
'led_163',
'led_164',
'led_165',
'led_166',
'led_167',
'led_168',
'led_169',
'led_170',
'led_171',
'led_172',
'led_173',
'led_174',
'led_175',
'led_176',
'led_177',
'led_178',
'led_179',
'led_180',
'led_181',
'led_182',
'led_183',
'led_184',
'led_185',
'led_186',
'led_189',
'led_190',
'led_191',
'led_192',
'led_193',
'led_194',
'led_195',
'led_196',
'led_197',
'led_198',
'led_199',
'led_200',
'led_201',
'led_202',
'led_203',
'led_204',
'led_205',
'led_206',
'led_207',
'led_208',
'led_209',
'led_210',
'led_211',
'led_212',
'led_213',
'led_214',
'led_215',
'led_216',
'led_217',
'led_218',
'led_219',
'led_220',
'led_221',
'led_222',
'led_223',
'led_224',
'led_225',
'led_226',
'led_227',
'led_228',
'led_229',
'led_230',
'led_231',
'led_232',
'led_233',
'led_234',
'led_235',
'led_236',
'led_237',
'led_238',
'led_239',
'led_240',
'led_241',
'led_242',
'led_243',
'led_244',
'led_245',
'led_246',
'led_247',
'led_248',
'led_249',
'led_250',
'led_251',
'led_252',
'led_253',
'led_254',
'led_255',
'led_256',
'led_257',
'led_258',
'led_259',
'led_260',
'led_261',
'led_262',
'led_265',
'led_266',
'led_267',
'led_268',
'led_269',
'led_270',
'led_271',
'led_272',
'led_273',
'led_274',
'led_275',
'led_276',
'led_277',
'led_278',
'led_279',
'led_280',
'led_281',
'led_282',
'led_283',
'led_284',
'led_285',
'led_286',
'led_287',
'led_288',
'led_289',
'led_290',
'led_291',
'led_292',
'led_293',
'led_294',
'led_295',
'led_296',
'led_297',
'led_298',
'led_299',
'led_300',
'led_301',
'led_302',
'led_303',
'led_304',
'led_305',
'led_306',
'led_307',
'led_308',
'led_309',
'led_310',
'led_311',
'led_312',
'led_313',
'led_314',
'led_315',
'led_316',
'led_317',
'led_318',
'led_319',
'led_320',
'led_321',
'led_322',
'led_323',
'led_324',
'led_325',
'led_326',
'led_327',
'led_328',
'led_329',
'led_330',
'led_331',
'led_332',
'led_333',
'led_334',
'led_335',
'led_336',
'led_337',
'led_338',
'led_339',
'led_340',
'led_341',
'led_342',
'led_343',
'led_344',
'led_345',
'led_346',
'led_347',
'led_348',
'led_349',
'led_350',
'led_351',
'led_352',
'led_353',
'led_354',
'led_355',
'led_356',
'led_357',
'led_358',
'led_359',
'led_360',
'led_361',
'led_362',
'led_363',
'led_364',
'led_365',
'led_366',
'led_367',
'led_368',
'led_369',
'led_370',
'led_371',
'led_372',
'led_373',
'led_374',
'led_375',
'led_376',
'led_377',
'led_378',
'led_379',
'led_380',
'led_381',
'led_382',
'led_383',
'led_384',
'led_385',
'led_386',
'led_387',
'led_388',
'led_389',
'led_390',
'led_391',
'led_392',
'led_393',
'led_394',
'led_395',
'led_396',
'led_397',
'led_398',
'led_399',
'led_400',
'led_401',
'led_402',
'led_403',
'led_404',
'led_405',
'led_406',
'led_407',
'led_408',
'led_409',
'led_410',
'led_411',
'led_412',
'led_413',
'led_414',
'led_415',
'led_416',
'led_417',
'led_418',
'led_419',
'led_420',
'led_421',
'led_422',
'led_423',
'led_424',
'led_425',
'led_426',
'led_427',
'led_428',
'led_429',
'led_430',
'led_431',
'led_432',
'led_433',
'led_434',
'led_435',
'led_436',
'led_437',
'led_438',
'led_439',
'led_440',
'led_441',
'led_442',
'led_443',
'led_444',
'led_445',
'led_446',
'led_447',
'led_448',
'led_449',
'led_450',
'led_451',
'led_452',
'led_453',
'led_454',
'led_455',
'led_456',
'led_457',
'led_458',
'led_459',
'led_460',
'led_461',
'led_462',
'led_463',
'led_464',
'led_465',
'led_466',
'led_467',
'led_468',
'led_469',
'led_470',
'led_471',
'led_472',
'led_473',
'led_474',
'led_475',
'led_476',
'led_477',
'led_478',
'led_479',
'led_480',
'led_481',
'led_482',
'led_483',
'led_484',
'led_485',
'led_486',
'led_487',
'led_488',
'led_489',
'led_490',
'led_491',
'led_492',
'led_493',
'led_494',
'led_495',
'led_496',
'led_497',
'led_498',
'led_499',
'led_500',
'led_501',
'led_502',
'led_503',
'led_504',
'led_505',
'led_506',
'led_507',
'led_508',
'led_509',
'led_510',
'led_511',
'led_512',
'led_513',
'led_514',
'led_515',
'led_516',
'led_517',
'led_518',
'led_519',
'led_520',
'led_521',
'led_522',
'led_523',
'led_524',
'led_525',
'led_526',
'led_527',
'led_528',
'led_529',
'led_530',
'led_531',
'led_532',
'led_533',
'led_534',
'led_535',
'led_536',
'led_537',
'led_538',
'led_539',
'led_540',
'led_541',
'led_542',
'led_543',
'led_544',
'led_545',
'led_546',
'led_547',
'led_548',
'led_549',
'led_550',
'led_551',
'led_552',
'led_553',
'led_554',
'led_555',
'led_556',
'led_557',
'led_558',
'led_559',
'led_560',
'led_561',
'led_562',
'led_563',
'led_564',
'led_565',
'led_566',
'led_567',
'led_568',
'led_569',
'led_570',
'led_571',
'led_572',
'led_573',
'led_574',
'led_575',
'led_576',
'led_577',
'led_578',
'led_579',
'led_580',
'led_581',
]